This notebook was inpired by this LlamaIndex notebook:

https://colab.research.google.com/drive/1GyPRMiwxS7rKxKpRt4r-ckYfmAw2GxdQ?usp=sharing

Making some changes to it with the only intention of trying ideas and learning.

Notice that I am assuming you have the relevant API_KEYs as environmental variables.

In [21]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import QueryEngineTool, ToolMetadata
import os
import subprocess

## Defining global variables

In [22]:
COMPANIES = ["uber"]
DATA_DIRS = {}
PERSIST_DIRS = {}
for c in COMPANIES:
    DATA_DIRS[c] = os.path.join(os.environ["DATA_DIR"], f"{c}_q")
    PERSIST_DIRS[c] = os.path.join(os.environ["PERSIST_DIR"], f"{c}_q")

## Download data
You can access more files by usinig the next url format

https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/{`COMPANY_NAME`}_10q_{`MONTH`}_{`YEAR`}.pdf'

In [26]:
for c in COMPANIES:
    if not os.path.exists(DATA_DIRS[c]):
        os.mkdir(DATA_DIRS[c])
        for year in ["2022"]:
            for month in ["march", "june", "sept"]:
                command = f"wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10q/{c}_10q_{month}_{year}.pdf' -O '{DATA_DIRS[c]}/{month}_{year}.pdf'"        
                subprocess.run(command, shell=True)

# Create index and query engine for each company individually

In [30]:
query_engine_dict = {}
for c in COMPANIES:
    if not os.path.exists(PERSIST_DIRS[c]):
        print("Creating Index")
        # load the documents and create the index
        documents = SimpleDirectoryReader(DATA_DIRS[c]).load_data()
        index = VectorStoreIndex.from_documents(documents)
        # store it for later
        index.storage_context.persist(persist_dir=PERSIST_DIRS[c])
    else:
        print("Loading Index")
        # load the existing index
        storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIRS[c])
        index = load_index_from_storage(storage_context)
    
    query_engine_dict[c] = index.as_query_engine(similarity_top_k=3)
    

Loading Index


## Define tool

In [31]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine_dict[c],
        metadata=ToolMetadata(
            name=f"{c}_{d}",
            description=(
                f"Provides information about {c} financials for {d}. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    )
    for c in COMPANIES
    for d in [f.split(".")[0] for f in os.listdir(DATA_DIRS[c])]
]

## Agent stepwise

In [40]:
llm_gpt4 = OpenAI(model="gpt-4")
gpt4_agent_worker = FunctionCallingAgentWorker.from_tools(query_engine_tools, llm=llm_gpt4, verbose=True, allow_parallel_tool_calls=False)
gpt4_agent = AgentRunner(gpt4_agent_worker)

In [19]:
task = gpt4_agent.create_task("Analyze the changes in R&D expenditures and revenue for Uber in June 2022 or March 2022")
completed = False
while not completed:
    step_output = gpt4_agent.run_step(task.task_id)
    completed = step_output.is_last
    if completed:
        print("Completed task")
    else:
        print("Task not completed, moving to next task.")


Added user message to memory: Analyze the changes in R&D expenditures and revenue for Uber in June 2022 or March 2022


=== Calling Function ===
Calling function: uber_june_2022 with args: {"input": "What were the R&D expenditures and revenue for Uber in June 2022?"}
=== Function Output ===
In June 2022, Uber's Research and Development expenditures were $704 million, representing 9% of revenue. The revenue for Uber in June 2022 was $8,073 million.
Task not completed, moving to next task.
=== Calling Function ===
Calling function: uber_march_2022 with args: {"input": "What were the R&D expenditures and revenue for Uber in March 2022?"}
=== Function Output ===
The Research and Development expenditures for Uber in March 2022 were $587 million. The revenue for Uber in March 2022 was $6,854 million.
Task not completed, moving to next task.
=== LLM Response ===
In March 2022, Uber's Research and Development (R&D) expenditures were $587 million, and the revenue was $6,854 million. 

By June 2022, the R&D expenditures increased to $704 million, representing a growth of approximately 20%. The revenue also increa

## Adding Human Feedback

In [41]:
task = gpt4_agent.create_task("Analyze the changes in R&D expenditures and revenue from march to june")

In [42]:
step_output = gpt4_agent.run_step(task.task_id)

Added user message to memory: Analyze the changes in R&D expenditures and revenue from march to june
=== Calling Function ===
Calling function: uber_march_2022 with args: {"input": "What were the R&D expenditures and revenue for March 2022?"}
=== Function Output ===
The Research and Development expenditures for March 2022 were $587 million, and the revenue for the same period was not explicitly provided in the context information.


In [43]:
step_output = gpt4_agent.run_step(task.task_id, input='What about September?')

Added user message to memory: What about September?
=== Calling Function ===
Calling function: uber_sept_2022 with args: {"input": "What were the R&D expenditures and revenue for September 2022?"}
=== Function Output ===
The Research and Development expenditures for September 2022 were $760 million, and the revenue for September 2022 was not explicitly provided in the context.
